In [ ]:
!git clone https://github.com/declare-lab/conv-emotion

Cloning into 'conv-emotion'...
remote: Enumerating objects: 1388, done.
remote: Counting objects: 100% (324/324), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 1388 (delta 156), reused 218 (delta 91), pack-reused 1064
Receiving objects: 100% (1388/1388), 751.31 MiB | 13.87 MiB/s, done.
Resolving deltas: 100% (700/700), done.
Checking out files: 100% (435/435), done.


In [ ]:
import pickle

_, videoSpeakers, videoLabels, _, _, _, videoSentence, trainVid, testVid = pickle.load(open("/content/drive/MyDrive/Depot_Mouhamed-20220228T100527Z-001/DialogueRNN/DialogueRNN/IEMOCAP_features/IEMOCAP_features_raw.pkl", 'rb'), encoding='latin1')



In [ ]:
print(len(list(videoSentence.values())[0]))
print(len(list(videoLabels.values())[0]))

42
42


In [ ]:
%cd /content/conv-emotion/TL-ERC/

/content/conv-emotion/TL-ERC


In [ ]:
!python setup.py

In [ ]:
!cp /content/drive/MyDrive/Depot_Mouhamed-20220228T100527Z-001/DialogueRNN/DialogueRNN/IEMOCAP_features/IEMOCAP_features_raw.pkl /content/conv-emotion/TL-ERC/datasets

In [ ]:
!gdown --id 1OXtnyJ5nDMmK75L9kEQvKPIyO0xzyeVC


Downloading...
From: https://drive.google.com/uc?id=1OXtnyJ5nDMmK75L9kEQvKPIyO0xzyeVC
To: /content/conv-emotion/TL-ERC/cornell_weights.pkl
100% 42.0M/42.0M [00:00<00:00, 78.0MB/s]


In [ ]:
%cd bert_model

/content/conv-emotion/TL-ERC/bert_model


In [ ]:
!python train.py --load_checkpoint=../generative_weights/cornell_weights.pkl --data=iemocap

In [ ]:
!gdown --id 1nufbrBJ-LtcROv1MviCHFI7tQE3JnqQR

Downloading...
From: https://drive.google.com/uc?id=1nufbrBJ-LtcROv1MviCHFI7tQE3JnqQR
To: /content/conv-emotion/TL-ERC/iemocap.zip
100% 14.2M/14.2M [00:00<00:00, 44.1MB/s]


In [ ]:
!unzip /content/conv-emotion/TL-ERC/iemocap.zip -d /content/conv-emotion/TL-ERC/datasets

Archive:  /content/conv-emotion/TL-ERC/iemocap.zip
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/._.DS_Store  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/.DS_Store  
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/test/
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/IEMOCAP_features_raw.pkl  
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/train/
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/video_id.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/sentence_length.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/sentences.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/labels.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/conversation_length.pkl  
  inflating: /content/conv-emotion/TL-ERC/dat

In [ ]:
from solver import *
from data_loader import get_loader
from configs import get_config
from util import Vocab
import os
import pickle


def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


config = get_config(parse = False, mode='train')
val_config = get_config(parse = False, mode='valid')
test_config = get_config(parse = False, mode='test')

# _RUNS = config.runs

# _best_test_loss, _best_test_f1_w, _best_test_f1_m, _best_epoch = [], [], [], []

# for run in range(_RUNS):

print(config)

# No. of videos to consider
training_data_len = int(config.training_percentage * \
    len(load_pickle(config.sentences_path)))


train_data_loader = get_loader(
    sentences=load_pickle(config.sentences_path)[:training_data_len],
    labels=load_pickle(config.label_path)[:training_data_len],
    conversation_length=load_pickle(config.conversation_length_path)[:training_data_len],
    sentence_length=load_pickle(config.sentence_length_path)[:training_data_len],
    batch_size=config.batch_size)

eval_data_loader = get_loader(
    sentences=load_pickle(val_config.sentences_path),
    labels=load_pickle(val_config.label_path),
    conversation_length=load_pickle(val_config.conversation_length_path),
    sentence_length=load_pickle(val_config.sentence_length_path),
    batch_size=val_config.eval_batch_size,
    shuffle=False)

test_data_loader = get_loader(
    sentences=load_pickle(test_config.sentences_path),
    labels=load_pickle(test_config.label_path),
    conversation_length=load_pickle(test_config.conversation_length_path),
    sentence_length=load_pickle(test_config.sentence_length_path),
    batch_size=test_config.eval_batch_size,
    shuffle=False)




# for testing
solver = Solver(config, train_data_loader,
                eval_data_loader, test_data_loader, is_train=True)


solver.build()

    # best_test_loss, best_test_f1_w, best_epoch = solver.train()

    # print(f"Current RUN: {run+1}")

    # print("\n\nBest test loss")
    # print(best_test_loss)
    # print("Best test f1 weighted")
    # print(best_test_f1_w)
    # print("Best epoch")
    # print(best_epoch)

    # _best_test_loss.append(best_test_loss)
    # _best_test_f1_w.append(best_test_f1_w)
    # _best_epoch.append(best_epoch)


# Print final
# print(f"\n\nAverage across runs:")

# print("Best epoch")
# print(_best_epoch)

# print("\n\nBest test loss")
# print(np.mean(np.array(_best_test_loss), axis=0))

# print("Overall test f1 weighted")
# print(np.array(_best_test_f1_w))

# print("Best test f1 weighted")
# print(np.mean(np.array(_best_test_f1_w), axis=0))

iemocap
iemocap
iemocap
Configurations
{'activation': 'Tanh',
 'batch_size': 2,
 'bidirectional': True,
 'checkpoint': None,
 'clip': 1.0,
 'context_size': 256,
 'conversation_length_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/conversation_length.pkl'),
 'data': 'iemocap',
 'data_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train'),
 'dataset_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap'),
 'dropout': 0.0,
 'embedding_size': 300,
 'encoder_hidden_size': 768,
 'eval_batch_size': 2,
 'feedforward': 'FeedForward',
 'id2word_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/id2word.pkl'),
 'label_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/labels.pkl'),
 'learning_rate': 0.0001,
 'load_checkpoint': None,
 'minimum_improvement': 0.001,
 'mode': 'train',
 'model': 'bc_RNN',
 'n_epoch': 500,
 'num_bert_layers': 4,
 'num_classes': 6,
 'num_layers': 1,
 'optimizer': <class 'torch.optim.adam.Ada

100%|██████████| 231508/231508 [00:00<00:00, 425012.28B/s]


Build Graph


100%|██████████| 407873900/407873900 [00:27<00:00, 14715835.60B/s]


Parameter initiailization
	context_encoder.rnn.weight_hh_l0
	encoder.embeddings.word_embeddings.weight True
	encoder.embeddings.position_embeddings.weight True
	encoder.embeddings.token_type_embeddings.weight True
	encoder.embeddings.LayerNorm.weight True
	encoder.embeddings.LayerNorm.bias True
	encoder.encoder.layer.0.attention.self.query.weight True
	encoder.encoder.layer.0.attention.self.query.bias True
	encoder.encoder.layer.0.attention.self.key.weight True
	encoder.encoder.layer.0.attention.self.key.bias True
	encoder.encoder.layer.0.attention.self.value.weight True
	encoder.encoder.layer.0.attention.self.value.bias True
	encoder.encoder.layer.0.attention.output.dense.weight True
	encoder.encoder.layer.0.attention.output.dense.bias True
	encoder.encoder.layer.0.attention.output.LayerNorm.weight True
	encoder.encoder.layer.0.attention.output.LayerNorm.bias True
	encoder.encoder.layer.0.intermediate.dense.weight True
	encoder.encoder.layer.0.intermediate.dense.bias True
	encoder.enc

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
validation_loss, w_valid_f1, valid_predictions = solver.evaluate(eval_data_loader, mode="valid")


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
labels = [e[1] for e in eval_data_loader]
Labels = [label for utt in labels for label in utt]
Labels = [e for k in Labels for e in k]

In [ ]:
Labels[0]

1

In [ ]:
solver.print_metric(Labels, valid_predictions, "test")

test
              precision    recall  f1-score   support

           0     0.0781    0.0391    0.0521       128
           1     0.0909    0.0133    0.0233        75
           2     0.0000    0.0000    0.0000       244
           3     0.1720    0.9674    0.2920       184
           4     0.0000    0.0000    0.0000       222
           5     0.0000    0.0000    0.0000       258

    accuracy                         0.1656      1111
   macro avg     0.0568    0.1700    0.0612      1111
weighted avg     0.0436    0.1656    0.0559      1111



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

0.0559376253553394

In [ ]:
test_loss, w_test_f1, test_predictions = solver.evaluate(test_data_loader, mode="test")

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 125 kB 4.1 MB/s 


In [ ]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 123 kB 4.2 MB/s 
     |████████████████████████████████| 132 kB 34.4 MB/s 
     |████████████████████████████████| 8.6 MB 49.8 MB/s 
     |████████████████████████████████| 79 kB 5.9 MB/s 
     |████████████████████████████████| 138 kB 54.4 MB/s 
     |████████████████████████████████| 127 kB 51.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


load or preprocess moseidata